## Description
____________
This script generates forcing files for ESP. Model Initialization date, ESP date ranges and HYPE forcing files  
are defined. Forcing ensembles are generated by extracting the initialization period forcings and appending  
values from the ESP period from each year in the record. These are saved as .nc files. 


### Import Libraries

In [ ]:
import os
import pandas as pd
from datetime import datetime
import xarray as xr
import numpy as np
import time

### Inputs

In [ ]:
# esp ensemble decisions
esp_year= ['2003', '2010']                 # years for esp forcings (minimum one year)
start_date= '04-01'                        # start month and day for esp analysis in %m-%d
end_date= '09-30'                          # end month and day for esp analysis in %m-%d

initialization_date= '01-01'               # start month and day for model spin up in %m-%d
previous_year_as_esp= True                 # if true, initialization_date will be taken as the year before the esp_year 
                                           # e.g. if true and esp_year=2010, initialization date will be 2009-%m-%d

forcing_files= ['Tobs.txt', 'Pobs.txt', 'TMAXobs.txt', 'TMINobs.txt'] # list of HYPE forcing files
hype_directory= '../model'                  # location of HYPE model files (default is ./model)

output_directory= '../esp_forcings'         # location of output directory, default is esp_forcings

Alternative Input

In [ ]:
# Create forcings for every year in a range (uncomment)
start_year= 1982 
end_year = 2015
esp_year = [str(year) for year in range(start_year, end_year + 1)]
esp_year

### Generate Ensemble

In [ ]:
# Record start time
start_time = time.time()

In [ ]:
# Save the current working directory path
current_directory = os.getcwd()

In [ ]:
# Extract month and day from the dates, excluding the year
start_month, start_day = start_date.split('-')
end_month, end_day = end_date.split('-')
initialization_month, initialization_day = end_date.split('-')

# Convert dates to integers
start_month, start_day = int(start_month), int(start_day)
end_month, end_day = int(end_month), int(end_day)
initialization_month, initialization_day = int(end_month), int(end_day)

In [ ]:
# Create the parent directory if it doesn't exist already
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Parent directory '{output_directory}' created successfully.")
else:
    print(f"Parent directory '{output_directory}' already exists.")
    

In [ ]:
# Iterate through each year in esp_year
for analysis_year in esp_year:
    
    # Define the output directory name for the current year
    year_directory = os.path.join(output_directory, f"{analysis_year}_forcings")
    
    # Create the results directory if it doesn't exist already
    if not os.path.exists(year_directory):
        os.makedirs(year_directory)
        print(f"Directory '{year_directory}' created successfully.")
    else:
        print(f"Directory '{year_directory}' already exists.")

    # Convert year to int
    analysis_year = int(analysis_year)

    
    for file_name in forcing_files:
        
        # Read forcing file
        os.chdir(hype_directory)
        
        forcing = pd.read_csv(file_name, index_col=0, sep='\t')

        # Set the time series in the index to datetime
        forcing.index = pd.to_datetime(forcing.index)
        
        if previous_year_as_esp == True:
            
            initialization_year= analysis_year - 1  # use the previous year
            
        else:

            initialization_year= analysis_year

        # Initialize dataframe for initial conditions of the analysis year
        initial_conditions = forcing.loc[f"{initialization_year}-{initialization_date}": f"{analysis_year}-{start_date}"]

        # Extract unique years from the datetime index to iterate through
        unique_years = forcing.index.year.unique()
        
        # Remove the start year if it exists in the unique_years list to not consider it in the ensemble
        if analysis_year in unique_years:
            unique_years = unique_years[unique_years != analysis_year]

        # Create an empty list to store ensemble_member DataFrames
        ensemble_member_list = []

        # Create the ensemble by iterating through each year
        for i, year in enumerate(unique_years):
            # find the esp period of the current year
            start_date_year = pd.Timestamp(year, start_month, start_day)
            end_date_year = pd.Timestamp(year, end_month, end_day)
            
            # Extract esp members for current year
            year_data = forcing.loc[start_date_year:end_date_year]
            
            # Change the year of the current year to the analysis year
            year_data.index = year_data.index.map(lambda x: x.replace(year=analysis_year))

            # Append the filtered data to the initial_conditions DataFrame
            ensemble_member = pd.concat([initial_conditions, year_data])

            # Create an xarray DataArray from the ensemble member
            data_array = xr.DataArray(ensemble_member.values,
                                      dims=('time', 'subbasin'),
                                      coords={'time': ensemble_member.index,
                                              'subbasin': ensemble_member.columns})

            # Assign ensemble_member dimension coordinate
            data_array.coords['ensemble_member'] = i

            # Append the DataArray to the list
            ensemble_member_list.append(data_array)
        
        os.chdir(current_directory)

        # Concatenate ensemble_member DataArrays along the ensemble_member dimension
        data_array = xr.concat(ensemble_member_list, dim='ensemble_member')

        # Save the DataArray to a netCDF file for the current year
        data_array.to_netcdf(f'{current_directory}/{year_directory}/{file_name.split(".")[0]}.nc')
        
    print(f'{analysis_year} forcings complete')

In [ ]:
# Record end time
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")